# BE/Bi 103, Fall 2016: Homework 4
## Due 1pm, Sunday, October 23

(c) 2016 Justin Bois. With the exception of the images, this work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained therein is licensed under an [MIT license](https://opensource.org/licenses/MIT).

*This homework was generated from an Jupyter notebook.  You can download the notebook [here](hw4.ipynb).*

In [1]:
import collections
import itertools
import pandas as pd
import scipy.stats as st
import statsmodels.tools.numdiff as smnd
import numpy as np
import numba
import random
import math
from copy import copy, deepcopy

random.seed()

### Problem 4.1: Writing your own MCMC sampler (60 pts + 40 pts extra credit)

**a)** Write your own MCMC sampler that employs a Metropolis-Hastings algorithm that uses a Gaussian proposal distribution. Since you are sampling multiple parameters, your proposal distribution will be multi-dimensional. You can use a Gaussian proposal distribution with a diagonal covariance. In other words, you generate a proposal for each variable in the posterior independently.

You can organize your code how you like, but here is a suggestion.

* Write a function that takes (or rejects) a Metropolis-Hastings step. It should look something like the below (obviously where it does something instead of `pass`ing).

In [2]:

def mh_step(x, log_post, log_post_current, sigma, args=()):
    """
    Parameters
    ----------
    x : ndarray, shape (n_variables,)
        The present location of the walker in parameter space.
    log_post : function
        The function to compute the log posterior. It has call
        signature `log_post(x, *args)`.
    log_post_current : float
        The current value of the log posterior.
    sigma : ndarray, shape (n_variables, )
        The standard deviations for the proposal distribution.
    args : tuple
        Additional arguments passed to `log_post()` function.

    Returns
    -------
    x_out : ndarray, shape (n_variables,)
        The position of the walker after the Metropolis-Hastings
        step. If no step is taken, returns the inputted `x`.
    log_post_updated : float
        The log posterior after the step.
    accepted : bool
        True is the proposal step was taken, False otherwise.
    """
    mu, inv_cov = args
    
    #Sample next point
    nextPoint = np.random.multivariate_normal(x, cov)

    #Calculate metropolis ratio
    logmetropolisRatio = log_post(nextPoint, *args) - log_post_current
    
    #This is converting the log metropolis ratio to an actual value between 0 and 1 for probability
    logMetropolisRatioEx = np.exp(log_post(nextPoint, *args))/ np.exp(log_post_current)
    
    #Random decimal between 0 and 1 to decide if we should proceed with new point at a certain probability
    n = random.uniform(0,1)
    
    #If metropolis ratio is >= 1 or the random n is less than the probability (meaning we proceed with the new point)
    #at a probability of logMetropolisRatioEx
    if (logmetropolisRatio >= 1 or n <= logMetropolisRatioEx):
        return nextPoint, log_post(np.array(nextPoint), *args), True
    else:
        return x, log_post_current, False

<div class="alert alert-info">Looks good. 20/20 </div>

* Write another function that calls that function over and over again to do the sampling. It should look something like this:

In [3]:
def mh_sample(log_post, x0, sigma, args=(), n_burn=1000, n_steps=5000,
              variable_names=None):
    """
    Parameters
    ----------
    log_post : function
        The function to compute the log posterior. It has call
        signature `log_post(x, *args)`.
    x0 : ndarray, shape (n_variables,)
        The starting location of a walker in parameter space.
    sigma : ndarray, shape (n_variables, )
        The standard deviations for the proposal distribution.
    args : tuple
        Additional arguments passed to `log_post()` function.
    n_burn : int, default 1000
        Number of burn-in steps.
    n_steps : int, default 1000
        Number of steps to take after burn-in.
    variable_names : list, length n_variables
        List of names of variables. If None, then variable names
        are sequential integers.
    
    Returns
    -------
    output : DataFrame
        The first `n_variables` columns contain the samples.
        Additionally, column 'lnprob' has the log posterior value
        at each sample.
    """
    samples = []
    lnprob = []
    finalPoint = x0
    new_mu, new_cov = args
    logPostCurrent = log_post(x0, *args)
    isAccepted = True
    
    #Burn-in period
    
    for i in range(n_burn):
        finalPoint, logPostCurrent, isAccepted = mh_step(finalPoint, log_post, logPostCurrent, sigma, args)
    
    #After burn-in, we actually log in sample and log posterior values
    for j in range(n_steps):
        finalPoint, logPostCurrent, isAccepted = mh_step(finalPoint, log_post, logPostCurrent, sigma, args)
        samples.append(finalPoint)
        lnprob.append(logPostCurrent)
    d = {'Samples': samples, 'Log Posterior Value': lnprob}
    df = pd.DataFrame(data=d)
    return df


<div class="alert alert-info">Also looks good. 20/20 </div>

**b)** To test your code, we will get samples out of a known distribution. We will use a bivariate Gaussian with a mean of $\boldsymbol{\mu} = (10, 20)$ and covariance matrix of 

\begin{align}
\boldsymbol{\sigma} = \begin{pmatrix}
4 & -2 \\
-2 & 6
\end{pmatrix}
\end{align}

I have written the function to be unnormalized and JITted with numba for optimal speed.

Do not be confused: In this test function we are sampling $\mathbf{x}$ out of $P(\mathbf{x}\mid \boldsymbol{\mu},\boldsymbol{\sigma})$. This is not sampling a posterior; it's just a test for your code. You will pass `log_test_distribution` as the `log_post` argument in the above functions.

In [4]:
mu = np.array([10.0, 20])
cov = np.array([[4, -2],[-2, 6]])
inv_cov = np.linalg.inv(cov)

@numba.jit(nopython=True)
def log_test_distribution(x, mu, inv_cov):
    """
    Unnormalized log posterior of a multivariate Gaussian.
    """
    return -np.dot((x-mu), np.dot(inv_cov, (x-mu))) / 2

In [5]:
_out = mh_sample(log_test_distribution, np.array([5, 15]), cov, args=(mu, inv_cov))
_out

,Log Posterior Value,Samples
0,-0.077520,"[10.7286131875, 19.3016183674]"
1,-0.077520,"[10.7286131875, 19.3016183674]"
2,-0.077520,"[10.7286131875, 19.3016183674]"
3,-1.408060,"[6.67185386951, 22.1487422519]"
4,-1.408060,"[6.67185386951, 22.1487422519]"
5,-1.024009,"[9.0356884497, 17.4692309232]"
6,-1.024009,"[9.0356884497, 17.4692309232]"
7,-0.134261,"[8.98804170429, 20.75605361]"
8,-0.134261,"[8.98804170429, 20.75605361]"
9,-0.017121,"[9.70440519007, 19.8988161759]"


In [6]:
#Variables for keeping track of totals for both means so we can get average later
totalFirst = 0
totalSecond = 0
count = 0

first = []
second = []
for i in _out['Samples']:
    first.append(i[0])
    second.append(i[1])
    totalFirst += i[0]
    totalSecond += i[1]
    count += 1
print ("First mean:", totalFirst/count)
print ("Second mean:", totalSecond/count)
np.cov(first, second)


First mean: 9.94122145874
Second mean: 20.1504886394


array([[ 4.23496647, -1.87220312],
       [-1.87220312,  6.07966491]])

Here, we see that the means and covariance from the MCMC closely resembles their true respective values.

<div class="alert alert-info">Looks like your test worked, but is a bit off. Also, would be nice to visualize convergence with a corner plot, or a plot of one of the walker's trajectories. Any ideas for how to get a more accurate solution. (Hint: why didn't you change your kwargs for `n_steps`?) 20/20 </div>

If you compute the means and covariance (using `np.cov()`) of your samples, you should come close to the inputed means and covariance. You might also want to plot your samples using `corner.corner()` to make sure everything makes sense.

**c)** (20 pts extra credit) Add in some logic to your Metropolis-Hastings sampler to enable *tuning*. This is the process of automatically adjusting the $\sigma$ in the proposal distribution such that the acceptance rate is desirable. The target acceptance rate is about 0.4. The developers of [PyMC3](https://github.com/pymc-devs/pymc3) use the scheme below, which is reasonable.

|Acceptance rate|Standard deviation adaptation|
|:---:|:-------------------:|
| < 0.001        |$\times$ 0.1|
|< 0.05         |$\times$ 0.5|
|< 0.2          |$\times$ 0.9|
|> 0.5          |$\times$ 1.1|
|> 0.75         |$\times$ 2|
|> 0.95         |$\times$ 10|

Be sure to test your code to demonstrate that it works.

**d)** (20 pts extra credit) Either adapt the functions you already wrote or write new ones to enable sampling of discrete variables. Again, be sure to test your code.

QUESTION 4.1, PART C

In [7]:
# CODE BELOW FOR PART C

def returnAcceptanceRate(log_post, x0, sigma, args=(), n_burn=1000, n_steps=5000,
              variable_names=None):
    """
    Parameters
    ----------
    log_post : function
        The function to compute the log posterior. It has call
        signature `log_post(x, *args)`.
    x0 : ndarray, shape (n_variables,)
        The starting location of a walker in parameter space.
    sigma : ndarray, shape (n_variables, )
        The standard deviations for the proposal distribution.
    args : tuple
        Additional arguments passed to `log_post()` function.
    n_burn : int, default 1000
        Number of burn-in steps.
    n_steps : int, default 1000
        Number of steps to take after burn-in.
    variable_names : list, length n_variables
        List of names of variables. If None, then variable names
        are sequential integers.
    
    Returns
    -------
    output : DataFrame
        The first `n_variables` columns contain the samples.
        Additionally, column 'lnprob' has the log posterior value
        at each sample.
    """
    finalPoint = x0
    new_mu, new_cov = args
    logPostCurrent = log_post(x0, *args)
    isAccepted = True
    
    #This is for fine tuning
    totalIsAccepted = 0;
    #Burn-in period
    
    for i in range(n_burn):
        finalPoint, logPostCurrent, isAccepted = mh_step(finalPoint, log_post, logPostCurrent, sigma, args)
    
    #After burn-in, we actually log in sample and log posterior values
    for j in range(n_steps):
        finalPoint, logPostCurrent, isAccepted = mh_step(finalPoint, log_post, logPostCurrent, sigma, args)
        if (isAccepted):
            totalIsAccepted += 1;
    return (totalIsAccepted/n_steps)


# This below updates the cov/sigma until we get a good enough acceptance rate for our data
returnAcceptanceRate1 = returnAcceptanceRate(log_test_distribution, np.array([5, 15]), cov , args=(mu, inv_cov))
while (returnAcceptanceRate1 < 0.38 or returnAcceptanceRate1 > 0.42):
    if returnAcceptanceRate1 < 0.001:
        cov = cov * 0.1
        returnAcceptanceRate1 = returnAcceptanceRate(log_test_distribution, np.array([5, 15]), cov , args=(mu, inv_cov))
    elif returnAcceptanceRate1 < 0.05:
        cov = cov * 0.5
        returnAcceptanceRate1 = returnAcceptanceRate(log_test_distribution, np.array([5, 15]), cov , args=(mu, inv_cov))
    elif returnAcceptanceRate1 < 0.38:
        cov = cov * 0.95
        returnAcceptanceRate1 = returnAcceptanceRate(log_test_distribution, np.array([5, 15]), cov , args=(mu, inv_cov))
    elif returnAcceptanceRate1 > 0.95:
        cov = cov * 10
        returnAcceptanceRate1 = returnAcceptanceRate(log_test_distribution, np.array([5, 15]), cov , args=(mu, inv_cov))
    elif returnAcceptanceRate1 > 0.75:
        cov = cov * 2
        returnAcceptanceRate1 = returnAcceptanceRate(log_test_distribution, np.array([5, 15]), cov , args=(mu, inv_cov))
    elif returnAcceptanceRate1 > 0.42:
        cov = cov * 1.05
        returnAcceptanceRate1 = returnAcceptanceRate(log_test_distribution, np.array([5, 15]), cov , args=(mu, inv_cov))
    else:
        break;

print ("The updated covariance matrix after adjustment to get quality acceptance rate")
cov

The updated covariance matrix after adjustment to get quality acceptance rate


array([[  8.73149835,  -4.36574918],
       [ -4.36574918,  13.09724753]])

<div class="alert alert-info">Just a note about copying and pasting code: Your doc-string is wrong now!  

This is an interesting approach to the tuning problem - it still helps you find an optimal $\sigma$ for the proposal distribution, but it's computationally intensive. You can see in Justin's solution that rather than tuning $\sigma$ only once, he checks the acceptance rate every 100 steps, and so tunes $\sigma$ (if necessary) every 100 steps. This way, he runs the MCMC only once, but still gets a good acceptance ratio overall. Your implementation has a much stricter requirement, as it must overall have a 0.4 acceptance ratio. (Note also that the point of tuning is to use your `n_steps` more effectively. Your approach would require you to run the simulation for 3 or 4 times that number of steps!)</div>

In [8]:
# Now, with the updated cov, we will run to see the acceptance rate.
returnAcceptanceRateFinal = returnAcceptanceRate(log_test_distribution, np.array([5, 15]), cov , args=(mu, inv_cov))
print ("The final acceptance rate after updating the cov is :", returnAcceptanceRateFinal)

The final acceptance rate after updating the cov is : 0.4182


In [9]:
_out = mh_sample(log_test_distribution, np.array([5, 15]), cov, args=(mu, inv_cov))

#Variables for keeping track of totals for both means so we can get average later
totalFirst = 0
totalSecond = 0
count = 0


first = []
second = []
for i in _out['Samples']:
    first.append(i[0])
    second.append(i[1])
    totalFirst += i[0]
    totalSecond += i[1]
    count += 1
print ("First mean:", totalFirst/count)
print ("Second mean:", totalSecond/count)
np.cov(first, second)


First mean: 9.97524318229
Second mean: 19.9745178116


array([[ 4.09419977, -2.00469546],
       [-2.00469546,  6.19378674]])

Closely following the scheme provided, we keep updating the cov matrix until we get an acceptance rate that is close to 0.4 as desired. As can be seen above, the final cov/sigma differs from the initial cov sent in, and the acceptance rate with the final cov/sigma is 0.408, awfully close to 0.4 which is good. Once again, we see that the MCMC calculated means and cov matrix resemble their true respective values.



<div class="alert alert-info">Good job testing that tuning works. You can also see that it's much more accurate than your results without tuning, using the same number of steps. 20/20 </div>

QUESTION 4.1, PART D

In [10]:
# CODE FOR PART D

def mh_step_discrete(x, log_post, log_post_current, sigma, args=()):
    """
    Parameters
    ----------
    x : ndarray, shape (n_variables,)
        The present location of the walker in parameter space.
    log_post : function
        The function to compute the log posterior. It has call
        signature `log_post(x, *args)`.
    log_post_current : float
        The current value of the log posterior.
    sigma : ndarray, shape (n_variables, )
        The standard deviations for the proposal distribution.
    args : tuple
        Additional arguments passed to `log_post()` function.

    Returns
    -------
    x_out : ndarray, shape (n_variables,)
        The position of the walker after the Metropolis-Hastings
        step. If no step is taken, returns the inputted `x`.
    log_post_updated : float
        The log posterior after the step.
    accepted : bool
        True is the proposal step was taken, False otherwise.
    """
    mu, inv_cov = args
    
    #Sample next point
    nextPoint = np.random.multivariate_normal(x, cov)

    #Calculate metropolis ratio
    logmetropolisRatio = log_post(nextPoint, *args) - log_post_current
    
    #This is converting the log metropolis ratio to an actual value between 0 and 1 for probability
    logMetropolisRatioEx = np.exp(log_post(nextPoint, *args))/ np.exp(log_post_current)
    
    #Random decimal between 0 and 1 to decide if we should proceed with new point at a certain probability
    n = random.uniform(0,1)
    
    #Because points must have discrete values, casting is used
    nextPoint = [int(round(nextPoint[0])), int(round(nextPoint[1]))]
    x = [int(round(x[0])), int(round(x[1]))]
    #If metropolis ratio is >= 1 or the random n is less than the probability (meaning we proceed with the new point)
    #at a probability of logMetropolisRatioEx
    if (logmetropolisRatio >= 1 or n <= logMetropolisRatioEx):
        return nextPoint, log_post(np.array(nextPoint), *args), True
    else:
        return x, log_post_current, False
    
def mh_sample_discrete(log_post, x0, sigma, args=(), n_burn=1000, n_steps=5000,
              variable_names=None):
    """
    Parameters
    ----------
    log_post : function
        The function to compute the log posterior. It has call
        signature `log_post(x, *args)`.
    x0 : ndarray, shape (n_variables,)
        The starting location of a walker in parameter space.
    sigma : ndarray, shape (n_variables, )
        The standard deviations for the proposal distribution.
    args : tuple
        Additional arguments passed to `log_post()` function.
    n_burn : int, default 1000
        Number of burn-in steps.
    n_steps : int, default 1000
        Number of steps to take after burn-in.
    variable_names : list, length n_variables
        List of names of variables. If None, then variable names
        are sequential integers.
    
    Returns
    -------
    output : DataFrame
        The first `n_variables` columns contain the samples.
        Additionally, column 'lnprob' has the log posterior value
        at each sample.
    """
    samples = []
    lnprob = []
    finalPoint = x0
    new_mu, new_cov = args
    logPostCurrent = log_post(x0, *args)
    isAccepted = True
    
    #Burn-in period
    
    for i in range(n_burn):
        finalPoint, logPostCurrent, isAccepted = mh_step_discrete(finalPoint, log_post, logPostCurrent, sigma, args)
    
    #After burn-in, we actually log in sample and log posterior values
    for j in range(n_steps):
        finalPoint, logPostCurrent, isAccepted = mh_step_discrete(finalPoint, log_post, logPostCurrent, sigma, args)
        samples.append(finalPoint)
        lnprob.append(logPostCurrent)
    d = {'Samples': samples, 'Log Posterior Value': lnprob}
    df = pd.DataFrame(data=d)
    return df

_out = mh_sample_discrete(log_test_distribution, np.array([5, 15]), cov, args=(mu, inv_cov))

#Variables for keeping track of totals for both means so we can get average later
totalFirst = 0
totalSecond = 0
count = 0

first = []
second = []
for i in _out['Samples']:
    first.append(i[0])
    second.append(i[1])
    totalFirst += i[0]
    totalSecond += i[1]
    count += 1
    
# Returning means that are discrete which calls for casting
print ("First mean:", int(round(totalFirst/count)))
print ("Second mean:", int(round(totalSecond/count)))
np.cov(first, second)



First mean: 10
Second mean: 20


array([[ 3.94800544, -2.01523841],
       [-2.01523841,  5.84869918]])

To take into account sampling of discrete variables, we merely have to cast the returned location of walker to an integer. When first attempting this remedy, we did a mere integer cast by itself, which essentially would always round down whatever value was getting ready to returned. Due to this, we were getting mean estimates that were always below what was inputted. To better use casting, we did an integer cast AFTER rounding the value to the nearest number. This ultimately outputted the means of 10 and 20 which resembles their true values.

<div class="alert alert-info">Useful analysis. Would have been nice to see the discrete distribution! Note that you could have simplified your code by using a kwarg like `discrete` that is true or false, or passing the version of `mh_step` that shoudlld be used. Also, can you explain why it's okay to compute the Metropolis ratio the same way, even though you've changed the proposal distribution by rounding? 15/20.</div>

<div class="alert alert-info">Final: 95/60 </div>